conda install squidpy scanpy rpy2 scvelo adjusttext r-randomforest
conda install entrain-spatial
pip install tangram-sc
pip install pypath-omnipath

In [9]:
import entrain as en
import anndata as ad
import scanpy as sc
import pandas as pd

Load Data and Visualize

In [10]:
# Zenodo Links
# velocity_adata_file = "/Users/wk/Github/entrain_vignette_data/ratz_atlas_velocities_sparse.h5ad"
# spatial_adata_file = "/Users/wk/Github/entrain_vignette_data/v11_vis.h5ad"
# ligand_target_matrix_file = "/Users/wk/Github/entrain_vignette_data/ligand_target_matrix_mm.csv"
velocity_adata_file = "ratz_atlas_velocities_sparse.h5ad"
spatial_adata_file = "v11_vis.h5ad"
ligand_target_matrix_file = "ligand_target_matrix_mm.csv"

adata = ad.read_h5ad(velocity_adata_file)
adata_st = ad.read_h5ad(spatial_adata_file)
ligand_target_matrix = pd.read_csv(ligand_target_matrix_file, index_col=0)


Cluster Velocities

In [ ]:
adata = en.cluster_velocities(adata, resolution = 0.02)
en.plot_velocity_clusters_python(adata,
                                 plot_file = "plot_velocity_clusters.png",
                                velocity_cluster_key = "velocity_clusters")


Recover Dynamics: Calculate velocity likelihoods via scvelo.


In [ ]:
adata = en.recover_dynamics_clusters(adata, 
                              n_jobs = 10,
                              return_adata = True, n_top_genes=None)


Run Entrain: The following function performs two steps: performs label transfer (via tangram) on the velocity clusters, followed by fitting a random forest model to the velocity likelihoods and ligand-target network,

In [ ]:
adata_result=en.get_velocity_ligands_spatial(adata,
                                             adata_st,
                                             organism="mouse",
                                             velocity_cluster_key = "velocity_clusters",
                                             ligand_target_matrix=ligand_target_matrix)


Plot

In [ ]:
en.plot_velocity_ligands_python(adata_result,
                                cell_palette="plasma",
                                velocity_cluster_palette = "black",
                                color="velocity_clusters",
                                plot_output_path = "plot_result1.png")

Step-by-step analysis: You may wish to perform label transfer separately from the ligand inference. For example if you would like to manually inspect the transferred labels before continuing.

In [ ]:
adata_st_transfer = en.velocity_label_transfer(adata,
                                               adata_st,
                                               plot="label_transfer_plot.png",
                                               organism="mouse",
                                               tangram_result_column = "velocity_label_transfer",
                                              velocity_cluster_key="velocity_clusters")

sc.pl.spatial(adata_st_transfer,
              color="velocity_label_transfer",
              save = "plot_labels.png")


In [9]:
adata_result = en.get_velocity_ligands_spatial(adata,
                                               adata_st = adata_st_transfer,
                                               tangram_result_column = "velocity_label_transfer",
                                               ligand_target_matrix = ligand_target_matrix,
                                               velocity_cluster_key = "velocity_clusters")
                                            

In [ ]:
annotation_key = "broadlabel"
adata = en.recover_dynamics_clusters(adata,
                                     n_jobs = 10,
                                     cluster_key = annotation_key,
                                     return_adata = True)

adata_result=en.get_velocity_ligands_spatial(adata,
                                             adata_st,
                                             organism="mouse",
                                             velocity_cluster_key = annotation_key,
                                             ligand_target_matrix=ligand_target_matrix)

en.plot_velocity_ligands_python(adata_result,
                                cell_palette="Set1",
                                velocity_cluster_palette = "black",
                                color = annotation_key,
                                plot_output_path = "plot_result3.png")